In [61]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers
from langchain_pinecone import PineconeVectorStore
import pinecone
from pinecone import Pinecone, ServerlessSpec

from uuid import uuid4
import os
import time
from dotenv import load_dotenv

In [53]:
load_dotenv()

True

In [40]:
# Etract data from PDF
def load_pdf(data):
   loader =  DirectoryLoader(path=data, 
                            glob='*.pdf',
                            loader_cls=PyPDFLoader)
   documents = loader.load()

   return documents

In [41]:
extracted_data = load_pdf('data/')
extracted_data

In [42]:
# Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, 
                                                chunk_overlap=20)
    text_chunks = text_splitter.split_documents(documents=extracted_data)
    
    return text_chunks

In [43]:
text_chunks = text_split(extracted_data)
len(text_chunks)
# text_chunks

10484


In [44]:
# Embedding model
def download_embeddings_model():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings



In [45]:
embeddings = download_embeddings_model()

/var/folders/l5/nfl8tw2j4jb7l_5plk_dxddr0000gn/T/ipykernel_37628/4235689176.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
/opt/homebrew/Caskroom/miniconda/base/envs/med-ast/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/Caskroom/miniconda/base/envs/med-ast/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'cached_download' (fro

In [46]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [47]:
query_result

[-0.03447728604078293,
 0.031023267656564713,
 0.006734997034072876,
 0.02610897831618786,
 -0.0393621064722538,
 -0.16030247509479523,
 0.066924087703228,
 -0.00644137617200613,
 -0.04745044559240341,
 0.014758842997252941,
 0.07087539881467819,
 0.05552750080823898,
 0.019193340092897415,
 -0.02625138871371746,
 -0.01010960340499878,
 -0.026940476149320602,
 0.022307490929961205,
 -0.022226663306355476,
 -0.149692565202713,
 -0.01749301701784134,
 0.007676353678107262,
 0.05435230955481529,
 0.003254495793953538,
 0.03172589838504791,
 -0.08462144434452057,
 -0.0294061116874218,
 0.05159566551446915,
 0.048124078661203384,
 -0.0033147772774100304,
 -0.058279260993003845,
 0.04196932166814804,
 0.022210663184523582,
 0.128188818693161,
 -0.022338958457112312,
 -0.011656227521598339,
 0.06292832642793655,
 -0.03287624567747116,
 -0.09122596681118011,
 -0.03117532841861248,
 0.05269956216216087,
 0.04703476279973984,
 -0.0842030718922615,
 -0.030056225135922432,
 -0.020744914188981056,


In [54]:
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_api_env = 'gcp-starter'

pc = Pinecone(api_key=pinecone_api_key)

In [57]:
index_name = 'medical-assistant-vector-db'

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [60]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [62]:
uuids = [str(uuid4()) for _ in range(len(text_chunks))]

vector_store.add_documents(documents=text_chunks, ids=uuids)

['926c7ec1-e06b-494f-8a48-aa5757214511',
 '1baf31dd-11ed-4575-acd0-559d13d1c466',
 'c5347036-bae2-4b8d-9602-554e67165587',
 'e8af020d-7414-47e9-b7a1-f3c661667252',
 'f63eb37f-83ed-4426-89ca-ce1f8719ed09',
 '453ee0d7-8770-4d75-bbce-ffbdce512033',
 'a59556bd-903c-4621-88a6-eb9140998e46',
 'b6b7006a-44a4-45f7-aaab-c7a14b017e90',
 '266ef86e-214b-40f4-a0d3-0525b90bdc01',
 '417b0892-d6cc-47db-b2c8-b0d28640473d',
 '4711cd50-05fd-4a50-a677-82be8f1ef51d',
 '9a9808cf-67a9-42f5-902e-de3a0055c877',
 '78066318-84d9-4ebe-bbf3-14fd27a7cce6',
 'fc85589a-4d4a-4bff-9f2d-481aa0ddfb81',
 'e44f3a4a-f9b2-477f-9568-e7c737ebe3c8',
 '0f4a7d03-e991-4f0b-8c7d-adaada3e2fbf',
 'c1a63acb-e991-40a4-8913-e86d11a7f091',
 '6a7cbde4-f497-4f85-bb5f-b2935bd151fa',
 '62dea8fe-09a7-4cab-b24d-d71d7b50decf',
 '7e820083-0064-4f48-aae7-2eef8f1fbf5b',
 'e3a4c342-6bdb-433e-84cc-1b3dcd9d8cac',
 '2d768d78-37c7-446d-ae09-fb74923c515b',
 '7c5df4aa-4064-4db3-89d8-a0c94cf9e3f8',
 '1991c149-c161-4114-844b-6dc5a7f720c0',
 'aa933bc6-4275-

In [63]:
results = vector_store.similarity_search(
    query="What are Allergies",
    k=2,
)

for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* GALE ENCYCLOPEDIA OF MEDICINE 2 2591Physical allergy [{'page': 297.0, 'source': 'data/gale_encyclopedia_of_medicine_vol4.pdf'}]
* KEY TERMS
Allergen —Any substance that irritates only those
who are sensitive (allergic) to it.
Asthma —Wheezing (labored breathing) due to
allergies or irritation of the lungs.
Decongestant —Medicines that shrink blood ves-
sels and consequently mucus membranes. Pseu-doephedrine, phenylephrine, and phenylpropano-lamine are the most common.
Sinus —Air-filled cavities surrounding the eyes and [{'page': 22.0, 'source': 'data/gale_encyclopedia_of_medicine_vol4.pdf'}]


In [64]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [65]:

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [67]:
llm = CTransformers(model="model/lllama-2-7b-chat.Q4_0.gguf",
                    model_type="llama",
                    config={'max_new_tokens':512,
                            'temperature':0.8})

RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-670639e5-6636b294189dc5a74a9698bc;6af008fc-88e0-426d-86b2-35499944058d)

Repository Not Found for url: https://huggingface.co/api/models/model/lllama-2-7b-chat.Q4_0.gguf/revision/main.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid username or password.

In [ ]:

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=vector_store.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)